In [67]:
import re
from keras.preprocessing.text import Tokenizer 

filename = "aesop.txt"

with open(filename, encoding='utf-8-sig') as f:
    text = f.read()
    
seq_length = 20
start_story = '|'*seq_length

# CLEANUP
text = text.lower()
text = start_story + text
text = text.replace('\n\n\n\n\n', start_story)

text = text.replace('\n', ' ')
text = re.sub(' +', '. ', text).strip()
text = text.replace('..', '.')

text = re.sub('([!"#$%&()*+,-/:;<=>?@[\]^_`{|}~])', r' \1', text)
text = re.sub('\s{2,}', ' ', text)

# Tokenization

tokenizer = Tokenizer(char_level = False, filters = '')
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1
token_list = tokenizer.texts_to_sequences([text])[0]

In [20]:
print(text[:200])

 | | | | | | | | | | | | | | | | | | | |the . fox . and . the . grapes . a . hungry . fox . saw . some . fine . bunches . of . grapes . hanging . from . a . vine . that . was . trained . along . a . h


In [24]:
import numpy as np
from keras.utils import np_utils

def generate_sequences(token_list, step):
    X = []
    y = []
    for i in range(0, len(token_list) - seq_length, step):
        X.append(token_list[i: i + seq_length])
        y.append(token_list[i + seq_length])
    
    y = np_utils.to_categorical(y, num_classes = total_words)
    
    num_seq = len(X)
    print('Number of sequences:', num_seq, "\n")
    
    return X, y, num_seq


In [25]:
step = 1
seq_length = 20
X, y, num_seq = generate_sequences(token_list, step)

X = np.array(X)
y = np.array(y)

Number of sequences: 92889 



In [43]:
from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.models import Model
from tensorflow.keras.optimizers import RMSprop

n_units = 256
embedding_size = 100

text_in = Input(shape = (None,))
x = Embedding(total_words, embedding_size)(text_in)
x = LSTM(n_units)(x)
x = Dropout(0.2)(x)
text_out = Dense(total_words, activation = 'softmax')(x)

model = Model(text_in, text_out)

opti = RMSprop(learning_rate = 0.001)
model.compile(loss='categorical_crossentropy', optimizer=opti)

epochs = 100
batch_size = 32
model.fit(X, y, epochs=epochs, batch_size=batch_size, shuffle = True)


Epoch 1/100
2903/2903 [==============================] - 102s 35ms/step - loss: 3.0859
Epoch 2/100
2903/2903 [==============================] - 102s 35ms/step - loss: 2.8642
Epoch 3/100
2903/2903 [==============================] - 100s 34ms/step - loss: 2.7808
Epoch 4/100
2903/2903 [==============================] - 100s 35ms/step - loss: 2.8116
Epoch 5/100
2903/2903 [==============================] - 102s 35ms/step - loss: 2.9500
Epoch 6/100
2903/2903 [==============================] - 97s 33ms/step - loss: 3.0765
Epoch 7/100
2903/2903 [==============================] - 97s 33ms/step - loss: 3.1462
Epoch 8/100
2903/2903 [==============================] - 97s 34ms/step - loss: 3.2026
Epoch 9/100
2903/2903 [==============================] - 98s 34ms/step - loss: 3.2454
Epoch 10/100
2903/2903 [==============================] - 98s 34ms/step - loss: 3.2541
Epoch 11/100
2903/2903 [==============================] - 744s 256ms/step - loss: 3.2567
Epoch 12/100
2903/2903 [=====================

In [104]:
def sample_with_temp(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probs = np.random.multinomial(1, preds, 1)
    return np.argmax(probs)

def generate_text(seed_text, next_words, model, max_sequence_len, temp):
    output_text = seed_text
    seed_text = start_story + seed_text
    
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences(seed_text)[0]
        token_list = token_list[-max_sequence_len:]
        token_list = np.reshape(token_list, (1, -1))

        probs = model.predict(token_list, verbose=0)[0]
        y_class = sample_with_temp(probs, temperature = temp)

        output_word = tokenizer.index_word[y_class] if y_class > 0 else ''

        if output_word == "|":
            break

        seed_text += output_word + ' '
        output_text += output_word + ' '
    return output_text

In [79]:
array = model.predict([[60,50,90]], verbose=0)[0]
len(array)

4893

In [80]:
print(len(tokenizer.word_index))

4892


In [128]:
generate_text('', 50, model, 400, 6.0)
# tokenizer.word_index

'land turned fox long - but you him which for their boar fell for which one , show all : search fell .gutenberg this how home had all a one after : some own : you their you appeared mouse if when . added mercury .c will gave she the '

In [113]:
model.save('aesop-model')

INFO:tensorflow:Assets written to: aesop-model/assets


INFO:tensorflow:Assets written to: aesop-model/assets
